IMPORTAÇÕES NECESSÁRIAS

In [1]:

import sys
import os
sys.path.append(os.path.abspath('../scripts'))

from myFunctions import install_packages, save_table 
install_packages()


Installing required packages: ['numpy', 'pandas', 'scikit-learn', 'joblib', 'pyarrow', 'fastparquet', 'plotly', 'matplotlib', 'MetaTrader5', 'tabulate', 'optuna', 'torch', 'tqdm']
numpy is already installed.
pandas is already installed.
scikit-learn is already installed.
joblib is already installed.
pyarrow is already installed.
fastparquet is already installed.
plotly is already installed.
matplotlib is already installed.
MetaTrader5 is already installed.
tabulate is already installed.
optuna is already installed.
torch is already installed.
tqdm is already installed.
All packages are verified.


In [7]:
### importing required packages
from tabulate import tabulate
import pandas as pd
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from tqdm import tqdm

from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import mean_squared_error, accuracy_score
from torch.utils.data import DataLoader, TensorDataset
import optuna
import optuna.visualization as vis
import os
import joblib

### MODEL CLASSES DEFINITION

In [8]:
# LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        lstm_out, (hn, cn) = self.lstm(x)
        out = self.fc(lstm_out[:, -1, :])  # Using the last output of the sequence
        return out

# GRU Model
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        gru_out, hn = self.gru(x)
        out = self.fc(gru_out[:, -1, :])  # Using the last output of the sequence
        return out

# CNN-LSTM Model
class CNNLSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout, conv_filters):
        super(CNNLSTMModel, self).__init__()
        self.conv1d = nn.Conv1d(input_size, conv_filters, kernel_size=3, padding=1)
        self.lstm = nn.LSTM(conv_filters, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = x.permute(0, 2, 1)  # For Conv1d, we need [batch, channels, seq_len]
        x = self.conv1d(x)
        x = x.permute(0, 2, 1)  # Returning to [batch, seq_len, channels]
        lstm_out, (hn, cn) = self.lstm(x)
        out = self.fc(lstm_out[:, -1, :])  # Using the last output of the sequence
        return out

# CNN-GRU Model
class CNNGRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout, conv_filters):
        super(CNNGRUModel, self).__init__()
        self.conv1d = nn.Conv1d(input_size, conv_filters, kernel_size=3, padding=1)
        self.gru = nn.GRU(conv_filters, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = x.permute(0, 2, 1)  # For Conv1d, we need [batch, channels, seq_len]
        x = self.conv1d(x)
        x = x.permute(0, 2, 1)  # Returning to [batch, seq_len, channels]
        gru_out, hn = self.gru(x)
        out = self.fc(gru_out[:, -1, :])  # Using the last output of the sequence
        return out


### PROCESSING DATA TO ARRAYS FOR WINDOWS SIZE AND LOOK FORWARD

In [9]:
def scale_features(df, exclude_columns=['date', 'day']):
    """
    Escalona todas as colunas do DataFrame, exceto as especificadas em exclude_columns.

    Parameters:
        df (pd.DataFrame): O DataFrame contendo os dados.
        exclude_columns (list): Lista de colunas que não serão escalonadas.

    Returns:
        df_scaled (pd.DataFrame): DataFrame com as colunas escalonadas.
        scalers (dict): Dicionário contendo os escaladores para cada coluna escalonada.
    """
    scalers = {}
    columns_to_scale = [col for col in df.columns if col not in exclude_columns]
    
    df_scaled = df.copy()
    for col in columns_to_scale:
        scaler = MinMaxScaler()
        df_scaled[col] = scaler.fit_transform(df[[col]])
        scalers[col] = scaler
    
    return df_scaled, scalers


def data_to_array(df, window_size, target, features):
    """
    Prepares X and y with targets shifted for the next day after the window.

    Parameters:
        df (pd.DataFrame): The dataframe containing the data.
        window_size (int): The window size (e.g., 7 days).
        target (str): The target column name (e.g., 'close_price_target').
        features (list): List of feature column names (e.g., ['open', 'high', 'low', 'close']).

    Returns:
        X (np.ndarray): Input features.
        y (np.ndarray): Target values.
        y_dates (np.ndarray): Dates associated with the targets.
    """
    X = []
    y = []
    y_dates = []

    for i in range(len(df) - window_size):
        # Access the target column directly by its name
        target_value = df.iloc[i + window_size][target]
        y.append(target_value)
        y_dates.append(df.iloc[i + window_size]['date'])
        
        # Prepare the features using the provided column names
        X.append(df.iloc[i:i + window_size][features].values)

    return np.array(X), np.array(y), np.array(y_dates)

SPLITING DATA TO TRAIN AND TEST


In [10]:
def segment_data(df, test_size=0.15):
    """
    Segments the data into training and test sets based on the test size percentage.
    """
    # Calculates the size of the test set
    test_len = int(len(df) * test_size)
    
    # Segments the data
    train_data = df[:-test_len]  # 85% for training
    test_data = df[-test_len:]   # 15% for testing
    
    return train_data,test_data

### Training and Testing Pipeline

In [16]:
def train_evaluate_model(trial, X_train, y_train, X_test, y_test, target, window_size, look_forward, model_type, study_name, model_dir):
    """
    Train and evaluate the model based on the suggested hyperparameters.
    """
    # Convert data to tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

    # Hyperparameter suggestions
    epochs = trial.suggest_int('epochs', 10, 20)
    batch_size = trial.suggest_categorical('batch_size', [32, 64])
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    dropout = trial.suggest_float('dropout', 0.2, 0.5)
    num_layers = trial.suggest_int('num_layers', 2, 3)
    hidden_size = trial.suggest_int('hidden_size', 16, 128)

    if model_type in ['CNN-LSTM', 'CNN-GRU']:
        conv_filters = trial.suggest_categorical('conv_filters', [32, 64, 128])

    # Model selection
    if model_type == 'LSTM':
        model = LSTMModel(input_size=X_train.shape[2], hidden_size=hidden_size, num_layers=num_layers, dropout=dropout)
    elif model_type == 'GRU':
        model = GRUModel(input_size=X_train.shape[2], hidden_size=hidden_size, num_layers=num_layers, dropout=dropout)
    elif model_type == 'CNN-LSTM':
        model = CNNLSTMModel(input_size=X_train.shape[2], hidden_size=hidden_size, num_layers=num_layers, dropout=dropout, conv_filters=conv_filters)
    elif model_type == 'CNN-GRU':
        model = CNNGRUModel(input_size=X_train.shape[2], hidden_size=hidden_size, num_layers=num_layers, dropout=dropout, conv_filters=conv_filters)

    # Optimizer and loss function
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss() if 'price' in target else nn.BCELoss()

    # DataLoader for training
    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=False)

    average_losses = {epoch: [] for epoch in range(epochs)}

    # Training loop
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            output = model(X_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()  # Accumulate the loss

        # Print loss for each epoch
        avg_loss = total_loss / len(train_loader)
        for epoch in range(epochs):
            average_losses[epoch].append(avg_loss)

    # Evaluation on test data
    model.eval()
    test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=batch_size, shuffle=False)
    predictions = []
    true_values = []
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            output = model(X_batch)
            predictions.append(output.numpy())
            true_values.append(y_batch.numpy())

    predictions = np.concatenate(predictions, axis=0)
    true_values = np.concatenate(true_values, axis=0)

    if 'price' in target:
        error = mean_squared_error(true_values, predictions)
    else:
        predictions = (predictions > 0.5).astype(int)
        error = -accuracy_score(true_values, predictions)

    loss_decay_file = os.path.join(model_dir, f"{study_name}_loss_decay.pkl")
    with open(loss_decay_file, 'wb') as f:
        joblib.dump(average_losses, f)

    return error


In [17]:
def optimize_models(df, targets, target_type, features, windows, look_forwards, max_samples=100):
    """
    Optimizes models using Optuna for hyperparameter tuning. Now prepares the data within the function.
    """
    study_results = {}
    input_dir = os.path.join('..', 'data', 'models', target_type).replace("/", "\\")  
    if not os.path.exists(input_dir):
        os.makedirs(input_dir)
    
    exclude_columns = ['date', 'day']
    df_scaled, _ = scale_features(df, exclude_columns)
    df_scaled, scalers = scale_features(df)
    train_data, test_data = segment_data(df_scaled, test_size=0.15)
    
    for window_size in windows:
        for look_forward in look_forwards:
            for target in targets:
                X_train, y_train, y_dates = data_to_array(train_data.copy(), 
                                                          window_size,
                                                          target, 
                                                          features)

                X_test, y_test, _ = data_to_array(test_data.copy(), 
                                                  window_size, 
                                                  target, 
                                                  features)

                model_names = ['LSTM', 'GRU', 'CNN-LSTM', 'CNN-GRU']
                for model_type in tqdm(model_names):
                    study_name = f"{model_type}_window_{window_size}_look_forward_{look_forward}_{target}"
                    model_dir = os.path.join(input_dir, study_name).replace("/", "\\")
                    
                    if not os.path.exists(model_dir):
                        os.makedirs(model_dir)
                    
                    # Creating the Optuna study
                    study = optuna.create_study(direction='minimize', study_name=study_name)
                    study.optimize(
                        lambda trial: train_evaluate_model(
                            trial, X_train, y_train, X_test, y_test, target, window_size, look_forward, model_type, study_name, model_dir
                        ),
                        n_trials=max_samples
                    )

                    study_file = os.path.join(model_dir, f"{study_name}_study.pkl").replace("/", "\\")
                    joblib.dump(study, study_file)
                    print(f'{study_name} saved to {study_file}')


                    best_params = study.best_params
                    best_trial_index = study.best_trial.number 
                    best_trial_value = study.best_value  

                    best_params_dict = {
                        'best_params': best_params,
                        'best_trial_index': best_trial_index,
                        'best_trial_value': best_trial_value
                    }
                    params_file = os.path.join(model_dir, f"{study_name}_best_params.pkl").replace("/", "\\")
                    with open(params_file, "wb") as f:
                        joblib.dump(best_params_dict, f)

                    try:
                        fig_optimization = vis.plot_optimization_history(study)
                        fig_optimization.write_image(os.path.join(model_dir, f"{study_name}_optimization_history.png"))

                        fig_importances = vis.plot_param_importances(study)
                        fig_importances.write_image(os.path.join(model_dir, f"{study_name}_param_importances.png"))

                        fig_slice = vis.plot_slice(study)
                        fig_slice.write_image(os.path.join(model_dir, f"{study_name}_slice_plot.png"))

                        print(f"Images saved to {model_dir}")
                    except Exception as e:
                        print(f"Error in plotting images for {study_name}: {e}")


                    print(f"Saved to {model_dir}:")
                    print(f"Best hyperparameters for {model_type}, window size: {window_size}, look forward: {look_forward}, target {target}: {best_params}")
                    print(f"Best trial index: {best_trial_index}, Best trial value: {best_trial_value}")

                    study_results[study_name] = {
                        "study": study,
                        "best_params": best_params,
                        "best_trial_index": best_trial_index,
                        "best_trial_value": best_trial_value,
                        "directory": model_dir
                    }

    return study_results

In [18]:

input_dir = os.path.join('..', 'data', 'processed')

features_name = 'daily_features.pkl'
input_dir_features = os.path.join('..', 'data', 'features')

df1 = pd.read_parquet(os.path.join(input_dir, 'df_daily.parquet')).replace("/", "\\")
df2 = pd.read_parquet(os.path.join(input_dir, 'df_timestamp.parquet')).replace("/", "\\")
daily_features = joblib.load(os.path.join(input_dir_features, 'daily_features.pkl'))
timnestamp_features = joblib.load(os.path.join(input_dir_features, '15min_timestamp_features.pkl'))

features = daily_features
targets = ['close_price_target', 'open_price_target', 'behavior_target']
windows = [7]#, 15, 30, 45, 60] 
look_forwards = [1]  

# # Call the optimize_models function
study_results = optimize_models(df1, targets,features, windows, look_forwards, max_samples=100)


  0%|          | 0/4 [00:00<?, ?it/s][I 2024-12-21 13:44:22,788] A new study created in memory with name: LSTM_window_7_look_forward_1_close_price_target
[I 2024-12-21 13:44:37,950] Trial 0 finished with value: 0.07875198125839233 and parameters: {'epochs': 11, 'batch_size': 32, 'learning_rate': 2.5069623247826513e-05, 'dropout': 0.4629577156784078, 'num_layers': 3, 'hidden_size': 48}. Best is trial 0 with value: 0.07875198125839233.
[I 2024-12-21 13:44:57,559] Trial 1 finished with value: 0.07919950783252716 and parameters: {'epochs': 17, 'batch_size': 64, 'learning_rate': 1.7722795463645685e-05, 'dropout': 0.4528624730382779, 'num_layers': 3, 'hidden_size': 90}. Best is trial 0 with value: 0.07875198125839233.
[I 2024-12-21 13:45:07,187] Trial 2 finished with value: 0.06899137049913406 and parameters: {'epochs': 18, 'batch_size': 64, 'learning_rate': 0.00010473126125547903, 'dropout': 0.22075493423979614, 'num_layers': 2, 'hidden_size': 29}. Best is trial 2 with value: 0.068991370499

LSTM_window_7_look_forward_1_close_price_target saved to ..\data\models\LSTM_window_7_look_forward_1_close_price_target\LSTM_window_7_look_forward_1_close_price_target_study.pkl


 25%|██▌       | 1/4 [37:00<1:51:02, 2220.89s/it][I 2024-12-21 14:21:23,683] A new study created in memory with name: GRU_window_7_look_forward_1_close_price_target


Images saved to ..\data\models\LSTM_window_7_look_forward_1_close_price_target
Saved to ..\data\models\LSTM_window_7_look_forward_1_close_price_target:
Best hyperparameters for LSTM, window size: 7, look forward: 1, target close_price_target: {'epochs': 18, 'batch_size': 32, 'learning_rate': 4.194763340764618e-05, 'dropout': 0.4792148808905295, 'num_layers': 3, 'hidden_size': 76}
Best trial index: 88, Best trial value: 0.043682295829057693


[I 2024-12-21 14:21:35,443] Trial 0 finished with value: 0.08163440227508545 and parameters: {'epochs': 13, 'batch_size': 64, 'learning_rate': 0.00013195123243419195, 'dropout': 0.4647796922968444, 'num_layers': 2, 'hidden_size': 81}. Best is trial 0 with value: 0.08163440227508545.
[I 2024-12-21 14:21:42,349] Trial 1 finished with value: 0.09828151762485504 and parameters: {'epochs': 11, 'batch_size': 64, 'learning_rate': 0.0016721499924803969, 'dropout': 0.32772823620797653, 'num_layers': 2, 'hidden_size': 127}. Best is trial 0 with value: 0.08163440227508545.
[I 2024-12-21 14:21:57,781] Trial 2 finished with value: 0.1279335916042328 and parameters: {'epochs': 19, 'batch_size': 32, 'learning_rate': 0.00033463700941438056, 'dropout': 0.38535348346283776, 'num_layers': 3, 'hidden_size': 89}. Best is trial 0 with value: 0.08163440227508545.
[I 2024-12-21 14:22:01,134] Trial 3 finished with value: 0.09155957400798798 and parameters: {'epochs': 10, 'batch_size': 64, 'learning_rate': 2.89

...

CNN-GRU_window_7_look_forward_1_open_price_target saved to ..\data\models\CNN-GRU_window_7_look_forward_1_open_price_target\CNN-GRU_window_7_look_forward_1_open_price_target_study.pkl


100%|██████████| 4/4 [2:14:01<00:00, 2010.32s/it]

Images saved to ..\data\models\CNN-GRU_window_7_look_forward_1_open_price_target
Saved to ..\data\models\CNN-GRU_window_7_look_forward_1_open_price_target:
Best hyperparameters for CNN-GRU, window size: 7, look forward: 1, target open_price_target: {'epochs': 16, 'batch_size': 32, 'learning_rate': 1.9224258459720607e-05, 'dropout': 0.2682226131007114, 'num_layers': 2, 'hidden_size': 126, 'conv_filters': 128}
Best trial index: 49, Best trial value: 0.03364561125636101



  0%|          | 0/4 [00:00<?, ?it/s][I 2024-12-21 18:02:54,812] A new study created in memory with name: LSTM_window_7_look_forward_1_behavior_target
[W 2024-12-21 18:02:54,877] Trial 0 failed with parameters: {'epochs': 14, 'batch_size': 32, 'learning_rate': 0.0014167584129807355, 'dropout': 0.45252862744755595, 'num_layers': 2, 'hidden_size': 38} because of the following error: RuntimeError('all elements of input should be between 0 and 1').
Traceback (most recent call last):
  File "c:\Users\guitz\anaconda3\envs\pytorch_env\lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\guitz\AppData\Local\Temp\ipykernel_26232\3815601039.py", line 40, in <lambda>
    lambda trial: train_evaluate_model(
  File "C:\Users\guitz\AppData\Local\Temp\ipykernel_26232\3586603527.py", line 48, in train_evaluate_model
    loss = criterion(output, y_batch)
  File "c:\Users\guitz\anaconda3\envs\pytorch_env\lib\site-packages\torch\nn\modu

RuntimeError: all elements of input should be between 0 and 1

In [ ]:
parametros = joblib.load()

In [ ]:
study_results

In [ ]:
df1.columns

In [151]:


models_dir = os.path.join('..', 'data', 'models', 'LSTM_window_7_look_forward_1_close_price_target_study.pkl').replace("/", "\\")

In [163]:
study = joblib.load('C:\\Users\\guitz\OneDrive\\Área de Trabalho\\monografia\\data\\models\\LSTM_window_7_look_forward_1_open_price_target\\LSTM_window_7_look_forward_1_open_price_target_study.pkl')

In [ ]:
parametros

In [99]:
x, y, label = data_to_array(df1, 7, 'close_price_target', features)

In [ ]:
X_train = torch.tensor(x, dtype=torch.float32)

In [ ]:
best_params = parametros.best_params
print(best_params)

In [ ]:
optimization_history = study.trials
for trial in optimization_history:
    print(f"Trial {trial.number}: Loss = {trial.value}, Params = {trial.params}")